### RunnableParallel in LangChain


`RunnableParallel` is a utility in LangChain that allows us to execute multiple tasks or chains in parallel. It is particularly useful when you need to perform independent operations simultaneously, such as validation, fraud detection, and compliance checks, as seen in this notebook.

### Tasks in This Notebook

1. **Validation**: A simple validation function will check the payment information to ensure the payment amount is valid.
2. **Fraud Detection**: Using a language model, the notebook will analyze payment transactions for potential fraud risks and provide a detailed assessment in JSON format.
3. **Compliance Check**: The notebook will perform a compliance check on payment transactions, listing any potential compliance concerns in a comma-separated format.
4. **Payment Processing Chain**: All the above tasks will be executed in parallel using `RunnableParallel`, and the results will be aggregated for further analysis.


In [ ]:
# from dotenv import load_dotenv

# load_dotenv('../env')

In [ ]:
from langchain_ollama import ChatOllama

base_url = "http://localhost:11434"
model = 'llama3.2:1b'
llm = ChatOllama(
    base_url=base_url,
    model = model,
    temperature = 0.8,
    num_predict = 256
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain.output_parsers import (
    CommaSeparatedListOutputParser, 
    StructuredOutputParser, 
    ResponseSchema
)

# Create output parsers
fraud_detection_parser = StructuredOutputParser.from_response_schemas([
    ResponseSchema(
        name="risk_level", 
        description="Fraud risk level (low/medium/high)"
    ),
    ResponseSchema(
        name="reasoning", 
        description="Brief explanation of fraud risk assessment"
    )
])

compliance_check_parser = CommaSeparatedListOutputParser()

# Modify prompts to include format instructions
fraud_detection_prompt = ChatPromptTemplate.from_template(
    "Analyze the payment transaction for fraud risk:\n"
    "Amount: ${amount}\n"
    "Merchant: {merchant}\n"
    "Card Type: {card_type}\n\n"
    "Provide a detailed fraud assessment in STRICT JSON format. Details need not be accurate. IMPORTANT: Ensure the output is a valid JSON object with EXACTLY these keys:\n"
    "{format_instructions}\n"
    "EXAMPLE FORMAT: {{\n"
    "  \"risk_level\": \"low\",\n"
    "  \"reasoning\": \"Detailed explanation here\"\n"
    "}}"
).partial(format_instructions=fraud_detection_parser.get_format_instructions())

compliance_check_prompt = ChatPromptTemplate.from_template(
    "Perform a compliance check on this payment transaction. "
    "Payment amount: ${amount}, Merchant: {merchant}, Country: {country}\n"
    "List any compliance concerns. Details need not be accurate. Respond in a comma-separated list:\n"
    "{format_instructions}"
).partial(format_instructions=compliance_check_parser.get_format_instructions())

# Validation function using a simple string output parser
def validate_payment(payment_info):
    if payment_info['amount'] <= 0:
        return "Validation Failed: Invalid payment amount"
    return "Validation Passed"

# Payment processing chain using RunnableParallel
payment_processing_chain = RunnableParallel({
    # Validation step
    "validation": RunnableLambda(validate_payment)
    ,
    
    # Fraud detection using LLM with output parser
     "fraud_detection": fraud_detection_prompt | llm | fraud_detection_parser
    ,

    # Compliance check using LLM with output parser
     "compliance_check": compliance_check_prompt | llm | compliance_check_parser
})

# Example usage
payment_info = {
    "amount": 5000,
    "merchant": "Global Tech Solutions",
    "card_type": "Corporate Credit Card",
    "country": "United States"
}

# Invoke the chain
result = payment_processing_chain.invoke(payment_info)
print("Payment Processing Results:")
for key, value in result.items():
    print(f"{key.replace('_', ' ').title()}: {value}")
